In [1]:
!pip install pandas numpy scikit-learn

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [4]:
file_path = '/content/Dataset .csv'  # Update path if needed
df = pd.read_csv(file_path)


In [14]:
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,73,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),1,0,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,73,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),1,0,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,75,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),1,0,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,75,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),0,0,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,75,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),1,0,No,No,4,4.8,Dark Green,Excellent,229


In [5]:
df.dropna(subset=['Cuisines', 'Aggregate rating', 'Average Cost for two', 'Votes'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
label_encoder = LabelEncoder()
df['City'] = label_encoder.fit_transform(df['City'])
df['Has Table booking'] = label_encoder.fit_transform(df['Has Table booking'])
df['Has Online delivery'] = label_encoder.fit_transform(df['Has Online delivery'])

In [7]:
vectorizer = TfidfVectorizer(stop_words='english')
cuisine_matrix = vectorizer.fit_transform(df['Cuisines'])

In [8]:
scaler = StandardScaler()
numerical_features = df[['Average Cost for two', 'Price range', 'Aggregate rating', 'Votes', 'Has Table booking', 'Has Online delivery']]
numerical_features_scaled = scaler.fit_transform(numerical_features)

In [9]:
features_combined = np.hstack((numerical_features_scaled, cuisine_matrix.toarray()))
cosine_sim = cosine_similarity(features_combined, features_combined)


In [10]:
# Step 9: Define recommendation function
def recommend_restaurants(restaurant_name, top_n=5):
    if restaurant_name not in df['Restaurant Name'].values:
        return "Restaurant not found. Please enter a valid restaurant name."

    index = df[df['Restaurant Name'] == restaurant_name].index[0]
    similarity_scores = list(enumerate(cosine_sim[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    recommended_indices = [i[0] for i in similarity_scores]

    return df.iloc[recommended_indices][['Restaurant Name', 'Cuisines', 'Aggregate rating', 'Average Cost for two', 'Votes']]

In [11]:
# Step 10: Test the recommendation system
sample_restaurant = 'Le Petit Souffle'  # Change as needed
recommendations = recommend_restaurants(sample_restaurant)
print(recommendations)

       Restaurant Name                          Cuisines  Aggregate rating  \
1     Izakaya Kikufuji                          Japanese               4.5   
1457           Kuuraku                          Japanese               3.9   
3800            Tamura                          Japanese               3.5   
2971       Boa Village  Chinese, Japanese, Korean, Asian               4.0   
1508          Daikichi                          Japanese               3.6   

      Average Cost for two  Votes  
1                     1200    591  
1457                  1250    106  
3800                  1500    163  
2971                  1500    225  
1508                  1000     70  


# Developed by Athulkrishna.KS